In [1]:
%load_ext autoreload  
%autoreload 2  
  
import random  
import numpy as np  
import torch  
import torch.nn as nn  
from braindecode import EEGClassifier

# dataset related  
from modules.competition_dataset import EEGDataset
from torch.utils.data import TensorDataset, DataLoader
from braindecode.models import EEGSimpleConv, EEGInceptionERP
from skorch.helper import predefined_split  
import random
import mne
from skorch.helper import predefined_split  
from skorch.callbacks import EpochScoring, Checkpoint, EarlyStopping, LRScheduler
from optuna.integration import SkorchPruningCallback
import optuna  
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device  

c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
data_path = './data/mtcaic3'
model_path = './checkpoints/ssvep/models/the_honored_one.pth'
optuna_db_path = './checkpoints/ssvep/optuna/the_honored_one.db'
eeg_channels = [
    "OZ", 
    "PO7",
    "PO8",
    "PZ",
]

# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [3]:
window_length = 1000 # ensure divisble by 64 the kernel size
stride = 20
batch_size = 64

dataset_train = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    data_fraction=1,
    task="SSVEP",
    eeg_channels=eeg_channels,
    tmin=1,
)

dataset_val = EEGDataset(
    data_path=data_path,
    window_length=window_length,
    stride=stride,
    split='validation',
    data_fraction=1,
    task="SSVEP",
    eeg_channels=eeg_channels,
    tmin=1,
)

X_train = dataset_train.data
y_train = dataset_train.labels

task: SSVEP, split: train, domain: time, data_fraction: 1
skipped: 114/2400
task: SSVEP, split: validation, domain: time, data_fraction: 1
skipped: 1/50


In [15]:
def set_seed(seed):    
    torch.manual_seed(seed)    
    torch.cuda.manual_seed_all(seed)    
    torch.backends.cudnn.deterministic = True    
    torch.backends.cudnn.benchmark = False    
    
def objective(trial):      
    set_seed(42 + trial.number)      
      
    # EEGInceptionERP hyperparameters - based on paper recommendations      
    n_filters = trial.suggest_int("n_filters", 4, 16, step=2)  # Default is 8  
    drop_prob = trial.suggest_float("drop_prob", 0.3, 0.7, step=0.1)  # Default is 0.5  
    depth_multiplier = trial.suggest_int("depth_multiplier", 1, 3)  # Default is 2  
    batch_norm_alpha = trial.suggest_float("batch_norm_alpha", 0.005, 0.05, log=True)  # Default is 0.01  
      
    # n_times - critical for SSVEP frequency resolution  
    n_times = trial.suggest_categorical("n_times", [500, 750, 1000, 1250])  # 2-5 seconds at 250Hz  
          
    # Activation function tuning      
    activation_name = trial.suggest_categorical("activation", ["ReLU", "ELU", "LeakyReLU"])      
    activation_map = {      
        "ReLU": torch.nn.ReLU,      
        "ELU": torch.nn.ELU,       
        "LeakyReLU": torch.nn.LeakyReLU,      
    }      
    activation = activation_map[activation_name]      
      
    # Training hyperparameters      
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)  
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])      
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)      
          
    # Scheduler parameters      
    use_scheduler = trial.suggest_categorical("use_scheduler", [True, False])      
    if use_scheduler:      
        scheduler_step_size = trial.suggest_int("scheduler_step_size", 10, 100, step=10)      
        scheduler_gamma = trial.suggest_float("scheduler_gamma", 0.1, 0.9, step=0.1)      
          
    # Early stopping patience      
    patience = trial.suggest_int("patience", 10, 50, step=5)      
          
    # Initialize EEGInceptionERP model with tuned parameters      
    model = EEGInceptionERP(      
        n_chans=4,      
        n_outputs=4,      
        n_times=n_times,  
        sfreq=250,  
        drop_prob=drop_prob,  
        n_filters=n_filters,  
        activation=activation,  
        batch_norm_alpha=batch_norm_alpha,  
        depth_multiplier=depth_multiplier,  
    )      
      
    import os      
    os.makedirs('./checkpoints/ssvep/optuna', exist_ok=True)  # Changed from mi to ssvep  
      
    callbacks = [      
        "accuracy",      
        SkorchPruningCallback(trial, monitor='valid_accuracy'),      
        Checkpoint(      
            f_params=f'./checkpoints/ssvep/optuna/best_model_trial_{trial.number}.pth',      
            monitor='valid_accuracy',      
            load_best=True,      
        ),      
        EarlyStopping(monitor='valid_accuracy', patience=patience, lower_is_better=False),      
    ]      
          
    # Setup scheduler if enabled      
    if use_scheduler:      
        callbacks.append(      
            LRScheduler(      
                policy='StepLR',      
                step_size=scheduler_step_size,      
                gamma=scheduler_gamma      
            )      
        )      
      
    clf = EEGClassifier(      
        model,      
        criterion=torch.nn.CrossEntropyLoss,      
        optimizer=torch.optim.Adam,      
        optimizer__lr=lr,      
        optimizer__weight_decay=weight_decay,      
        batch_size=batch_size,      
        max_epochs=200,      
        train_split=predefined_split(dataset_val),      
        device="cuda" if torch.cuda.is_available() else "cpu",      
        verbose=0,      
        callbacks=callbacks,      
    )      
      
    clf.fit(dataset_train)      
    return clf.history[-1]['valid_accuracy']
     
  
# Rest of optimization code remains the same  
study = optuna.create_study(    
    direction="maximize",    
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10, interval_steps=1)    
)    
    
study.optimize(objective, n_trials=50)  
    
print("\nOptimization finished.")    
print("Best trial:")    
trial = study.best_trial    
    
print(f"  Value: {trial.value}")    
print("  Params: ")    
for key, value in trial.params.items():    
    print(f"    {key}: {value}")    

[I 2025-06-30 20:30:51,119] A new study created in memory with name: no-name-e1b2c33c-2008-4027-8da2-620a8be8cf75
[W 2025-06-30 20:30:51,182] Trial 0 failed with parameters: {'n_filters': 6, 'drop_prob': 0.7, 'depth_multiplier': 1, 'batch_norm_alpha': 0.029182221378362842, 'n_times': 500, 'activation': 'ELU', 'lr': 2.239897647164504e-05, 'batch_size': 128, 'weight_decay': 0.005459507930918567, 'use_scheduler': True, 'scheduler_step_size': 20, 'scheduler_gamma': 0.5, 'patience': 25} because of the following error: RuntimeError('mat1 and mat2 shapes cannot be multiplied (128x124 and 60x4)').
Traceback (most recent call last):
  File "c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ahmad\AppData\Local\Temp\ipykernel_17096\1558568738.py", line 93, in objective
    clf.fit(dataset_train)
  File "c:\Users\ahmad\AppData\Local\Program

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x124 and 60x4)

In [ ]:
n_chans = 4
n_outputs = 4
n_convs = 2
kernel_size = 8
feature_maps = 112
resampling_freq = 100
activation = torch.nn.ELU
model = EEGSimpleConv(    
        n_chans=n_chans,    
        n_outputs=n_outputs,    
        sfreq=250,
        feature_maps=feature_maps,    
        n_convs=n_convs,    
        kernel_size=kernel_size,    
        resampling_freq=resampling_freq, 
        activation=activation, 
    )    
    
clf = EEGClassifier(  
    model,  
    criterion=torch.nn.CrossEntropyLoss,  
    optimizer=torch.optim.Adam,  
    optimizer__lr=2.13537735805541e-05,  
    optimizer__weight_decay=0.0002624302515969059,  
    batch_size=128,  
    max_epochs=20, # Or a reduced number for a final training run  
    train_split=predefined_split(dataset_val), # You might want to combine train+val here for final training  
    device="cuda" if torch.cuda.is_available() else "cpu",  
    verbose=1,  
    callbacks=["accuracy"],  
)  

try:
    clf.initialize()  
    clf.module_.load_state_dict(torch.load('./checkpoints/ssvep/eegsimpleconv.pth'))
except Exception:
    print('no module to load')
    
clf.fit(dataset_train.data, dataset_train.labels)
torch.save(clf.module_.module_.state_dict(), './checkpoints/ssvep/eegsimpleconv.pth')

no module to load
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      dur
-------  ----------------  ------------  -----------  ----------------  ------------  -------
      1            0.2690        1.4422       0.3162            0.3162        1.3425  74.0789
      2            0.2659        1.4267       0.3235            0.3235        1.3471  74.2054
      3            0.2777        1.4181       0.3108            0.3108        1.3520  73.8193
      4            0.2813        1.4110       0.2878            0.2878        1.3569  73.5667
      5            0.2862        1.4032       0.2201            0.2201        1.3606  73.5271
      6            0.2871        1.3990       0.2376            0.2376        1.3625  73.7118
      7            0.2853        1.3944       0.3525            0.3525        1.3647  73.6788
      8            0.2808        1.3914       0.3507      

In [ ]:
model = EEGInceptionERP(  
    n_chans=4,  
    n_outputs=4,  
    n_times=1000,  # 4 seconds at 250 Hz  
    sfreq=250,  
    drop_prob=0.5,  # Optimal dropout rate  
    n_filters=8,    # Default from paper  
    activation=torch.nn.ELU,  # Your current activation is good  
)
    
for i in range(100):
    clf = EEGClassifier(  
        model,  
        criterion=torch.nn.CrossEntropyLoss,  
        optimizer=torch.optim.Adam,  
        optimizer__lr=2.13537735805541e-05,  
        optimizer__weight_decay=0.0002624302515969059,  
        batch_size=128,  
        max_epochs=20, # Or a reduced number for a final training run  
        train_split=predefined_split(dataset_val), # You might want to combine train+val here for final training  
        device="cuda" if torch.cuda.is_available() else "cpu",  
        verbose=1,  
        callbacks=["accuracy"],  
    )  

    try:
        clf.initialize()  
        clf.module_.load_state_dict(torch.load('./checkpoints/ssvep/eeginception.pth'))
    except Exception:
        print('no module to load')
        
    clf.fit(X_train, y_train)
    torch.save(clf.module_.state_dict(), './checkpoints/ssvep/eeginception.pth')

no module to load
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.


c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Convolution.cpp:1037.)
  return F.conv2d(


  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      dur
-------  ----------------  ------------  -----------  ----------------  ------------  -------
      1            0.2803        1.4425       0.3029            0.3029        1.4467  60.5429
      2            0.2782        1.4213       0.2400            0.2400        1.3795  74.5871
      3            0.2805        1.4118       0.2793            0.2793        1.3743  74.6079
      4            0.2841        1.4053       0.2007            0.2007        1.3788  63.2287
      5            0.2831        1.3978       0.2678            0.2678        1.3805  32.9193
      6            0.2752        1.3933       0.2715            0.2715        1.3780  32.0307
      7            0.2790        1.3911       0.2715            0.2715        1.3756  32.7290
      8            0.2823        1.3895       0.2703            0.2703        1.3778  33.1047
      9            0.2855        1.3882       0.2727        